**MetaData Manager application For databases**  \
Using python , tkinter and MySQL
# **Nima Kelidari - 98108124** # 
**Database course**

**Step 0 :**
Import Libraries

In [7]:
import mysql.connector
import tkinter as tk
from tkinter import ttk, simpledialog

**Step 1 :**
Connect to MySQL Server

In [8]:



mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="#Yourpassword"
)
print(mydb)

**Step 2 :**
Creat Database as *DATABASE_COURSE*

In [9]:
mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE DATABASE_COURSE")

**Step 3 :**
Set mydb Varieble as *DATABASE_COURSE* database in MySQL server
, Then set mycrusor 

In [10]:
mydb = mysql.connector.connect(
  host="127.0.0.1",
  user="root",
  password="#Yourpassword",
  database="DATABASE_COURSE"
)
mycursor = mydb.cursor()

**Step 4 :**
Define some functions to creat the schema for storing a database. Then running them.

In [11]:
def create_schemas_table():
    mycursor.execute("""
    CREATE TABLE Schemas_table (
        schema_id INT NOT NULL AUTO_INCREMENT,
        schema_name VARCHAR(255) NOT NULL,
        PRIMARY KEY (schema_id)
    );
    """)
    
def create_tables_table():
    mycursor.execute("""
    CREATE TABLE Tables_table (
        table_id INT NOT NULL AUTO_INCREMENT,
        schema_id INT,
        table_name VARCHAR(255) NOT NULL,
        PRIMARY KEY (table_id)
    );
    """)

def create_columns_table():
    mycursor.execute("""
    CREATE TABLE Columns_table (
        column_id INT NOT NULL AUTO_INCREMENT,
        table_id INT,
        column_name VARCHAR(255) NOT NULL,
        data_type TEXT NOT NULL,
        is_nullable VARCHAR(3),
        key_value VARCHAR(255),
        default_value VARCHAR(255),
        extra TEXT,
        PRIMARY KEY (column_id)
    );
    """)

def create_Indexes_table():
    mycursor.execute("""
    CREATE TABLE Indexes_table (
        index_id INT NOT NULL AUTO_INCREMENT,
        table_id INT,
        column_id INT,
        non_unique INT,
        index_name VARCHAR(255) NOT NULL,
        seq_in_index INT UNSIGNED,  
        collation VARCHAR(1),
        cardinality BIGINT,
        sub_part BIGINT,
        packed TEXT,
        is_nullable VARCHAR(3),
        index_type VARCHAR(50),
        comment VARCHAR(255),
        index_comment VARCHAR(255),
        visible VARCHAR(255),
        expression TEXT,
        PRIMARY KEY (index_id) 
    );
    """)

def create_constraints_table():
    mycursor.execute("""
    CREATE TABLE Constraints_table (
    constraint_id INT NOT NULL AUTO_INCREMENT,
    table_id INT,
    column_id INT,
    con_name VARCHAR(255),
    ref_column_id INT,
    ref_table_id INT,
    con_type VARCHAR(255),
    PRIMARY KEY (constraint_id) 
);
""")

def define_foreign_keys():
    mycursor.execute("ALTER TABLE Tables_table ADD CONSTRAINT FOREIGN KEY (schema_id) REFERENCES Schemas_table(schema_id);")
    mycursor.execute("ALTER TABLE Columns_table ADD CONSTRAINT FOREIGN KEY (table_id) REFERENCES Tables_table(table_id);")
    mycursor.execute("ALTER TABLE Indexes_table ADD CONSTRAINT FOREIGN KEY (table_id) REFERENCES Tables_table(table_id);")
    mycursor.execute("ALTER TABLE Indexes_table ADD CONSTRAINT FOREIGN KEY (column_id) REFERENCES Columns_table(column_id);")
    mycursor.execute("ALTER TABLE Constraints_table ADD CONSTRAINT FOREIGN KEY (column_id) REFERENCES Columns_table(column_id);")
    mycursor.execute("ALTER TABLE Constraints_table ADD CONSTRAINT FOREIGN KEY (ref_column_id) REFERENCES Columns_table(column_id);")
    mycursor.execute("ALTER TABLE Constraints_table ADD CONSTRAINT FOREIGN KEY (table_id) REFERENCES Tables_table(table_id);")
    mycursor.execute("ALTER TABLE Constraints_table ADD CONSTRAINT FOREIGN KEY (ref_table_id) REFERENCES Tables_table(table_id);")






In [12]:
create_schemas_table()
create_tables_table()
create_columns_table()
create_Indexes_table()
create_constraints_table()
define_foreign_keys()


**Step 5 :**
Define connection manager class

In [13]:
class DatabaseManager:
    def __init__(self, host, user, password, database):
        self.connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        self.cursor = self.connection.cursor()

    def execute_query(self, query, values=None):
        if values:
            self.cursor.execute(query, values)
        else:
            self.cursor.execute(query)
        self.connection.commit()

    def fetch_data(self, query, values=None):
        if values:
            self.cursor.execute(query, values)
        else:
            self.cursor.execute(query)
        return self.cursor.fetchall()

    def insert_row(self, table_name, values,keys):
        insert_keys = str(keys)
        insert_keys = insert_keys.replace("[","").replace("]","").replace("\'","")
        query = f"INSERT INTO {table_name} ({insert_keys})  VALUES ({', '.join(['%s']*len(values))});"
        self.execute_query(query, values)

    def update_row(self, table_name, primary_key, new_values,keys):
        
        set_values = ', '.join([f"{column} = %s" for column in new_values.keys()])
        query = f"UPDATE {table_name} SET {set_values} WHERE {keys} = %s;"
        self.execute_query(query, list(new_values.values()) + [primary_key])


**Step 6 :**
Define Application class

In [14]:
class DatabaseApp:
    def __init__(self, root, host, user, password, database):
        self.root = root
        self.root.title("Database Management App")
        self.db_manager = DatabaseManager(host, user, password, database)
        self.main_menu()

    def main_menu(self):

        tabs = {
            "Schemas_table": ["schema_id", "schema_name"],
            "Tables_table": ["table_id", "schema_id", "table_name"],
            "Columns_table": ["column_id", "table_id", "column_name", "data_type", "is_nullable", "key_value","default_value","extra"],
            "Indexes_table": ["index_id","table_id","column_id","non_unique","index_name","seq_in_index","collation","cardinality",
            "sub_part","packed","is_nullable","index_type","expression"],
            "Constraints_table": ["constraint_id","table_id","column_id", "con_name", "ref_column_id", "ref_table_id","con_type"]
        }

        root.title("Database Metadata Manager")
        menu_bar = tk.Menu(root)

        file_menu = tk.Menu(menu_bar, tearoff=0)
        menu_bar.add_cascade(label="File", menu=file_menu)
        file_menu.add_command(label="Import database",command=lambda : self.import_db(tabs))
        file_menu.add_command(label="Export database",command=lambda : self.create_tabs("Schemas_table",tabs['Schemas_table'],'CHOOSE'))

        root.config(menu=menu_bar)

        


        button0 = tk.Button(self.root, text="Schemas", 
                    command=lambda: self.create_tabs("Schemas_table",tabs['Schemas_table']))
        button1 = tk.Button(self.root, text="Tables",    
                    command=lambda: self.create_tabs("Tables_table",tabs['Tables_table']))
        button2 = tk.Button(self.root, text="Columns", 
                    command=lambda: self.create_tabs("Columns_table",tabs['Columns_table']))
        button3 = tk.Button(self.root, text="Indexes", 
                    command=lambda: self.create_tabs("Indexes_table",tabs['Indexes_table']))
        button4 = tk.Button(self.root, text="Constraints",
                    command=lambda: self.create_tabs("Constraints_table",tabs['Constraints_table']))


        button0.pack(pady=20,padx=200)
        button1.pack(pady=20,padx=200)
        button2.pack(pady=20,padx=200)
        button3.pack(pady=20,padx=200)
        button4.pack(pady=20,padx=200)


    def create_tabs(self,tab_name,columns,mode=''):
        
        self.root = tk.Tk()
        self.root.title('Table Manager')
        tab_control = ttk.Notebook(self.root)

        tab_frame = ttk.Frame(tab_control)
        tab_control.add(tab_frame, text=tab_name)

        tree = ttk.Treeview(tab_frame, columns=columns, show="headings")
        for col in columns:
            tree.heading(col, text=col)
            tree.column(col, width=100)

        tree.grid(row=0, column=0, sticky="nsew")

        scroll_y = ttk.Scrollbar(tab_frame, orient="vertical", command=tree.yview)
        scroll_y.grid(row=0, column=1, sticky="ns")
        tree.configure(yscrollcommand=scroll_y.set)

        if tab_name == "Indexes_table":
            columns = ["index_id","table_id","column_id","non_unique","index_name","seq_in_index","collation","cardinality",
                "sub_part","packed","is_nullable","index_type","comment","index_comment","visible","expression"]

        if mode:
            choose_button = tk.Button(tab_frame, text="Select database", command=lambda tab_name=tab_name: self.export_database(tree, tab_name, columns[1:]))
            choose_button.grid(row=1, column=0, padx=10, pady=5)

        else:
            add_button = tk.Button(tab_frame, text="Add Row", command=lambda tab_name=tab_name: self.add_row_dialog(tree, tab_name, columns[1:]))
            add_button.grid(row=1, column=0, padx=10, pady=5)

            update_button = tk.Button(tab_frame, text="Update Row", command=lambda: self.update_row_dialog(tree, tab_name,columns))
            update_button.grid(row=1, column=1, padx=10, pady=5)

            remove_button = tk.Button(tab_frame, text="Remove Row", command=lambda: self.remove_row(tree, tab_name,columns[0]))
            remove_button.grid(row=1, column=2, padx=10, pady=5)

        self.populate_tree(tree, tab_name)

        tab_control.pack(expand=1, fill="both")

    def populate_tree(self, tree, table_name):
        data = self.db_manager.fetch_data(f"SELECT * FROM {table_name};")
        for row in data:
            tree.insert("", "end", values=row)

    def import_db(self,tabs):
        indexes_columns = ["table_id","column_id","non_unique","index_name","seq_in_index","collation","cardinality",
            "sub_part","packed","is_nullable","index_type","comment","index_comment","visible","expression"]
        connection_info = {}
        connection_info['host'] = simpledialog.askstring("Host Address","Enter host address:")
        connection_info['user'] = simpledialog.askstring("Username", "Enter username:")
        connection_info['password'] = simpledialog.askstring("Pasword","Enter password:")
        connection_info['db_name'] = simpledialog.askstring("Database Name","Enter Database_name:")
        to_import_db = mysql.connector.connect(
            host=connection_info['host'],
            user=connection_info['user'],
            password=connection_info['password'],
            database=connection_info['db_name']
            )
        imp = to_import_db.cursor()
        ###########
        self.db_manager.insert_row("Schemas_table", [connection_info['db_name']],tabs['Schemas_table'][1:])
        schema_id = self.db_manager.fetch_data(f"Select schema_id From Schemas_table Where schema_name = \'{connection_info['db_name']}\'")[0][0]
        ###########
        imp.execute("SHOW TABLES")
        table_names = [table[0] for table in imp.fetchall()]
        ###########
        for table_name in table_names:
            
            self.db_manager.insert_row("Tables_table",[schema_id,table_name],tabs['Tables_table'][1:])
            table_id = self.db_manager.fetch_data(f"Select table_id From Tables_table Where table_name = \'{table_name}\'")[0][0]
            imp.execute(f"DESCRIBE {table_name}")
            columns = [column for column in imp.fetchall()]
            ###########
            for column in columns:
                column = list(column)
                column.insert(0,table_id)
                self.db_manager.insert_row("Columns_table",column,tabs['Columns_table'][1:])
            ###########
            imp.execute(f"SHOW INDEXES FROM {table_name}")
            indexes = [column for column in imp.fetchall()]
            for index in indexes:
                index = list(index)
                column_name = index[4]
                column_id = self.db_manager.fetch_data(f"Select column_id From Columns_table Where table_id = \'{table_id}\' AND column_name = \'{column_name}\'")[0][0]
                index.pop(4)
                index.pop(0)
                index.insert(0,column_id)
                index.insert(0,table_id)
                self.db_manager.insert_row("Indexes_table",index,indexes_columns)
            ###########
        for table_name in table_names:
            table_id = self.db_manager.fetch_data(f"Select table_id From Tables_table Where table_name = \'{table_name}\'")[0][0]   
            imp = to_import_db.cursor(buffered=True,dictionary=True)
            imp.execute(f"""select COLUMN_NAME, CONSTRAINT_NAME, REFERENCED_COLUMN_NAME, REFERENCED_TABLE_NAME
                            from information_schema.KEY_COLUMN_USAGE
                            where TABLE_NAME = \'{table_name}\';""")
            to_import_db.commit()
            containers = imp.fetchall()
            for i,container in enumerate(containers):
                #if i==len(containers)//3:break
                cont = list(container.values())
                if cont[2] == None :
                    col_id1 = self.db_manager.fetch_data(f"Select column_id From Columns_table Where column_name = \'{cont[0]}\' AND table_id = \'{table_id}\'")[0][0]
                    cont.pop(0)
                    cont.insert(0,col_id1)
                    if cont[1] == 'PRIMARY':
                        cont.append('PRIMARY')
                    else:
                        cont.append('UNIQUE')
                else:
                    column1 = cont[0]
                    column2 = cont[2]
                    table1 = cont[3]
                    col_id1 = self.db_manager.fetch_data(f"Select column_id From Columns_table Where column_name = \'{column1}\' AND table_id = \'{table_id}\'")[0][0]
                    table_id1 = self.db_manager.fetch_data(f"Select table_id From Tables_table Where table_name = \'{table1}\'")[0][0]
                    col_id2 = self.db_manager.fetch_data(f"Select column_id From Columns_table Where column_name = \'{column2}\' AND table_id = \'{table_id1}\'")[0][0]
                    cont.pop(3)
                    cont.pop(2)
                    cont.pop(0)
                    cont.insert(0,col_id1)
                    cont.insert(2,col_id2)
                    cont.insert(3,table_id1)
                    cont.append('FOREIGN')
                cont.insert(0,table_id)
                self.db_manager.insert_row("Constraints_table",cont,tabs['Constraints_table'][1:])
        tk.messagebox.showinfo("IMPORTED", f"'{connection_info['db_name']}' database imported. Please check tables.") 

            
    def export_database(self, tree, table_name, columns):
        new_name = 'clone_company'
        new_name = simpledialog.askstring("Input Name","Choose a name for new database clone")
        selected_item = tree.selection()
        if not (selected_item and new_name):
            return
        row_data = tree.item(selected_item)['values']
        schema_id = row_data[0]
        connection_info = {}
        connection_info['host'] = simpledialog.askstring("Host Address","Enter host address:")
        connection_info['user'] = simpledialog.askstring("Username", "Enter username:")
        connection_info['password'] = simpledialog.askstring("Pasword","Enter password:")
        ###############
        to_export_db = mysql.connector.connect(
            host=connection_info['host'],
            user=connection_info['user'],
            password=connection_info['password']
            )
        exp = to_export_db.cursor()
        exp.execute(f"CREATE SCHEMA {new_name}")
        ###############
        to_export_db = mysql.connector.connect(
            host=connection_info['host'],
            user=connection_info['user'],
            password=connection_info['password'],
            database=new_name
            )
        exp = to_export_db.cursor()
        tables = self.db_manager.fetch_data(f"SELECT * FROM Tables_table where schema_id = {schema_id}")
        ###############
        for table in tables:
            table_id = table[0]
            table_name = table[2]
            query = "CREATE TABLE "+str(table_name) + " ( "
            columns = self.db_manager.fetch_data(f"SELECT * FROM Columns_table where table_id = {table_id}")
            for column in columns:
                query += " " + str(column[2]) + " " + str(column[3])
                if column[4]=='YES': query += " NOT NULL "
                query += ","
            constrains = self.db_manager.fetch_data(f"SELECT * FROM Constraints_table where table_id = {table_id}")
            pk = ' PRIMARY KEY ( '
            uq = ' UNIQUE ( '
            for constrain in constrains:
                con_type = constrain[6]
                if con_type == "PRIMARY":
                    col_name = self.db_manager.fetch_data(f"SELECT column_name FROM Columns_table where column_id = {constrain[2]}")[0][0]
                    pk+= str(col_name) + " ,"
                elif con_type == "UNIQUE":
                    col_name = self.db_manager.fetch_data(f"SELECT column_name FROM Columns_table where  column_id = {constrain[2]}")[0][0]
                    uq+= str(col_name) + " ,"
            if pk != ' PRIMARY KEY ( ':
                pkl = list(pk)
                pkl[-1] =')'
                pk = "".join(pkl)
                query += pk + ","
            if uq != ' UNIQUE ( ':
                uql = list(uq)
                uql[-1] =')'
                uq = "".join(uql)
                query += uq + ","
            queryl = list(query)
            queryl[-1]=')'
            query = "".join(queryl)
            query+=';'
            #print(query)
            exp.execute(query)

        ###############

        for table in tables:
            table_id = table[0]
            table_name = table[2]
            constrains = self.db_manager.fetch_data(f"SELECT * FROM Constraints_table where table_id = {table_id}")
            for constrain in constrains:
                con_type = constrain[6]
                if con_type == "FOREIGN":
                    con_name = constrain[3]
                    table_name2 = self.db_manager.fetch_data(f"SELECT table_name FROM Tables_table where table_id = {constrain[5]}")[0][0]
                    col_name1 = self.db_manager.fetch_data(f"SELECT column_name FROM Columns_table where column_id = {constrain[2]}")[0][0]
                    col_name2 = self.db_manager.fetch_data(f"SELECT column_name FROM Columns_table where column_id = {constrain[4]}")[0][0]
                    #print(f"ALTER TABLE {table_name} ADD CONSTRAINT {con_name} FOREIGN KEY ({col_name1}) REFERENCES {table_name2}({col_name2});")
                    exp.execute(f"ALTER TABLE {table_name} ADD CONSTRAINT {con_name} FOREIGN KEY ({col_name1}) REFERENCES {table_name2}({col_name2});")

        tk.messagebox.showinfo("EXPORTED", f"{row_data[1]} database exported as {new_name} database imported. Please check your host.")
        #exp.execute(f"DROP SCHEMA {new_name}")
 
        

    def add_row_dialog(self, tree, table_name, columns):
        new_values = {}
        for col in columns:
            value = simpledialog.askstring(f"Add Row - {table_name}", f"Enter value for {col}:")
            if value is not None:
                new_values[col] = value

        if new_values:
            self.db_manager.insert_row(table_name, list(new_values.values()),columns)
            self.refresh_tree(tree, table_name)

    def update_row_dialog(self, tree, table_name,columns):
        selected_item = tree.selection()
        if not selected_item:
            return

        row_data = tree.item(selected_item)['values']
        new_values = {}
        for col, value in zip(columns[1:], row_data[1:]):
            new_value = simpledialog.askstring(f"Update Row - {table_name}", f"Enter new value for {col} (current value: {value}):")
            if new_value is not None:
                new_values[col] = new_value

        if new_values:
            key = columns[0]
            primary_key = row_data[0]
            self.db_manager.update_row(table_name, primary_key, new_values,key)
            self.refresh_tree(tree, table_name)

    def remove_row(self, tree, table_name,id_col):
        selected_item = tree.selection()
        if selected_item:
            row_data = tree.item(selected_item)['values']
            self.db_manager.execute_query(f"DELETE FROM {table_name} WHERE {id_col} = %s;", (row_data[0],))
            self.refresh_tree(tree, table_name)

    def refresh_tree(self, tree, table_name):
        for item in tree.get_children():
            tree.delete(item)
        
        self.populate_tree(tree, table_name)




**Step 7 :**
Establish connection and **run application!**

In [15]:
if __name__ == "__main__":
    root = tk.Tk()
    app = DatabaseApp(root, host="127.0.0.1", user="root", password="#Yourpassword", database="DATABASE_COURSE")
    root.mainloop()
    app.db_manager.connection.close()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Nikel\AppData\Local\Programs\Python\Python39\lib\site-packages\mysql\connector\connection_cext.py", line 555, in cmd_query
    self._cmysql.query(
_mysql_connector.MySQLInterfaceError: Can't create database 'clone'; database exists

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\Nikel\AppData\Local\Programs\Python\Python39\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Nikel\AppData\Local\Temp\ipykernel_12356\307535763.py", line 76, in <lambda>
    choose_button = tk.Button(tab_frame, text="Select database", command=lambda tab_name=tab_name: self.export_database(tree, tab_name, columns[1:]))
  File "C:\Users\Nikel\AppData\Local\Temp\ipykernel_12356\307535763.py", line 211, in export_database
    exp.execute(f"CREATE SCHEMA {new_name}")
  File "c:\Users\Nikel\AppData\Local\Programs\Pyth